In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt


def epi_analysis(
        stripe_files, epi_names, epi_path_root, outdir,
        resolution=10000, region_size=1500000,
        chroms=None,
        colors=['blue', 'red', 'orange', 'green']# None: all chromosomes
):
    if chroms is None:
        chroms = [f'chr{elm}' for elm in list(range(1, 23)) + ['X']]
    length = region_size // resolution

    # Get stripe list
    stripes = {}
    counts = {}  # count the number of stripes, e.g., counts = {'Zebra': 100, 'Quagga': 200}
    for name in stripe_files:
        stripes[name] = {}
        counts[name] = 0
        f1 = open(stripe_files[name])
        for line in f1:
            # Skip the header line
            if line.startswith('#') or 'pos1' in line:
                continue
            lst = line.strip().split()
            ch, x1, x2, y1, y2 = lst[0], int(lst[1]), int(lst[2]), int(lst[4]), int(lst[5])
            # x1, x2, y1, y2 define a rectangle region
            # The longer edge of the rectangle region is the length of the stripe
            # The shorter edge of the rectangle region is the anchor of the stripe
            if x2 - x1 > y2 - y1:
                center = (y1 + y2) // 2 // resolution  # center of the anchor
                length_center = (x1 + x2) // 2 // resolution  # center of the "length of the stripe"
            else:
                length_center = (y1 + y2) // 2 // resolution
                center = (x1 + x2) // 2 // resolution
            # If the "center of the length" (i.e., the spanning direction) is upstream of the anchor,
            # then it's vertical
            if length_center < center:
                drc = 'v'
            else:
                drc = 'h'

            # fix chromosome names
            if not ch.startswith('chr'):
                ch = 'chr' + ch
            if ch not in stripes[name]:
                stripes[name][ch] = []
            stripes[name][ch].append((drc, center))
            counts[name] += 1

    # Initialize the empty vectors for storing epigenomic signals
    vecs = {name: {epi: np.zeros((length,)) for epi in epi_names} for name in stripe_files}
    # vecs = {'Quagga': {'CTCF': np.zeros((1000,)), 'RAD21': np.zeros((1000,))}, 'Zebra': {xxxxx} }

    for ch in chroms:
        print(ch)
        for epi in epi_names:
            print(' ', epi)
            # This is the path for epigenomic signals
            # H1 data are in the same format under xxx/Epi/H1/{ch}/
            # These data are all from ENCODE, data source is
            # see helper script "process_for_epi.py" under Quagga_Analysis/notebooks/

            epi_vec = np.load(f"{epi_path_root}{ch}/{ch}_500bp_{epi}.npy")
            # The resolution of the .npy files are 500 bp, we have to downsample to 10 Kb
            fc = resolution // 500
            epi_vec2 = np.zeros((len(epi_vec) // fc,))
            for i in range(len(epi_vec) // fc):
                epi_vec2[i] = np.sum(epi_vec[i * fc: (i + 1) * fc])

            for name in stripe_files:  # Quagga, Zebra, StripeNN
                if ch not in stripes[name]:
                    continue
                for (drc, pos) in stripes[name][ch]:
                    # Get the vector centered at the anchor point
                    # If the center point is too close to the start/end of the chromosome,
                    # we have to pad zeros
                    if pos < length // 2:
                        actual_length = pos + length // 2
                        vec = np.concatenate([np.zeros((length - actual_length),), epi_vec2[:actual_length]])
                    elif pos + length // 2 > len(epi_vec2):
                        actual_length = len(epi_vec2) - pos + length // 2
                        vec = np.concatenate([epi_vec2[pos - length // 2:], np.zeros((length - actual_length,))])
                    else:
                        vec = epi_vec2[pos - length // 2: pos + length // 2]
                    # If the stripe is vertical, reverse the epigenomic vector
                    # So all stripes in the APA analysis are "horizontal stripes"
                    if drc == 'v':
                        vec = vec[::-1]
                    vecs[name][epi] += vec

    if not os.path.exists(outdir):
        os.mkdir(outdir)

    colors = colors
        
    for epi in epi_names:
        plt.figure(figsize=(3.3*len(colors), 2),tight_layout=True)
        max_val = 0
        for name in stripe_files:
            vec = vecs[name][epi] / counts[name]
            max_val = max(max_val, max(vec))

        for i, name in enumerate(stripe_files.keys()):
            vec = vecs[name][epi] / counts[name]
            
            aset=1.0
            if colors[i] == "blue" or colors[i] == "red" or colors[i]=="orange":
                aset=0.4
            
            plt.subplot(1, len(colors), i + 1)
            plt.fill_between(np.arange(len(vec)), 0, vec, color=colors[i], alpha=aset)#color='#DB3124', alpha=0.8)
            # plt.ylabel('H - Forward')
            plt.ylim([-max_val * 0.05, max_val * 1.1])
            # plt.xticks([0.5, region_radius + 0.5, 2 * region_radius + 0.5],
            #            [f'-{region_size // 1000} Kb', 'Anchor', f'{region_size // 1000} Kb'])
            plt.xticks([0, length // 2, length], [f'-{region_size // 1000} Kb', 'Anchor', f'{region_size // 1000} Kb'])
            # plt.yticks([0, 0.1, 0.2, 0.3, 0.4, 0.5], [])

        plt.savefig(f'{outdir}/{epi}.png')
        plt.show()
   
                    

In [ ]:
colors=['blue']*3+['red']*3+['orange']*3
print(colors)

epi_names = [
    'H3K4me2', 'H3K4me3', 'H3K27ac', 'CTCF', 'RAD21'
]

epi_path_rootname = "/scratch/drjieliu_root/drjieliu/fanfeng/CAESAR_temp/training_data/Epi/GM12878/"

samples = {
    'Quagga All': './stripe_results/5000nt_MR10000000_ML50000_MD7000000_MW50000_WS7_SG1RH0.3_NSTR10_ALL_hg38GM12878_completedfiltered.txt',
    'Quagga CTCF': 'Quagga_GM12878_CTCFonly.txt',
    'Quagga EP': 'Quagga_GM12878_EP.txt',
    'Zebra All': './stripe_results/results_filtered.tsv',
    'Zebra CTCF': 'Zebra_AB_Merged_GM12878_CTCFonly.txt',
    'Zebra EP': 'Zebra_AB_Merged_GM12878_EP.txt',
    'StripeNN All': './stripe_results/GM12878.bedpe',
    'StripeNN CTCF': 'StripeNN_GM12878_CTCFonly.txt',
    'StripeNN EP': 'StripeNN_GM12878_EP.txt'

}




epi_analysis(samples, epi_names, epi_path_rootname, "GM12878_epigenome_pileups", resolution=500, region_size=200000,colors=colors)

In [ ]:
colors=['blue','royalblue', 'cornflowerblue', 'lightsteelblue','red','maroon','red','lightpink']
print(colors)

epi_names = [
    'H3K4me2', 'H3K4me3', 'H3K27ac', 'CTCF', 'RAD21'
]

epi_path_rootname = "/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/notebooks/figures/epigenomic/H1/"

samples = {
    'Hi-C Quagga All': "./stripe_results/H1_microC_vs_HiC/5000nt_MR15000000_ML20000_MD5000000_MW50000_WS10_SG1RH0.1_NSTR0_ALL_CHR.hg38H1.filtered.txt",
    'Hi-C Quagga CTCF': 'H1-hESC_HiC_CTCFonly.txt',
    'Hi-C Quagga Unassigned': 'Quagga_H1_HiC_Unassigned.txt',
    'Hi-C Quagga EP': 'H1-hESC_HiC_EP.txt',
    'Micro-C Quagga All': "./stripe_results/H1_microC_vs_HiC/5000nt_MR5000000_ML20000_MD500000_WS20_MW_50000_SG6RH0.1_NSTR10_alt_ALL_H1_microC_hg38_filtered.txt",
    'Micro-C Quagga CTCF': "H1-hESC_MiC_CTCFonly.txt",
    'Micro-C Quagga Unassigned': "Quagga_H1_MiC_Unassigned.txt", 
    'Micro-C Quagga EP': "H1-hESC_MiC_EP.txt",

}




epi_analysis(samples, epi_names, epi_path_rootname, "hic_vs_microC_H1_epigenome_pileups", resolution=500, region_size=200000,colors=colors)

In [ ]:
colors=['blue']*2

epi_names = [
    'H3K27ac', 'CTCF'
]

epi_path_rootname = "/nfs/turbo/umms-drjieliu/usr/temp_smoran/Fan_StripeCaller/070324_GIST-T1_RCMC/DATA/OUTPATH/Epi/"

samples = {
    'RCMC Quagga All': "./stripe_results/regioncapture_MicroC/MR5000000_ML20000_MD500000_MW10000_WS10_SG1_RH0.2_NSTR10_RCMC_GIST-T1_HG38_mcool_ALL.txt",
}



epi_analysis(samples, epi_names, "RCMC_GIST-T1_epigenomic", resolution=500, region_size=200000, colors=colors)